In [67]:
# Imports
import os
import requests
import pandas as pd
import numpy as np
from pathlib import Path
from dotenv import load_dotenv
from datetime import datetime
import hvplot.pandas
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report
import  yfinance as yf
# To show a progress bar when doing lengthy operations
from tqdm.notebook import tqdm

In [2]:
# Get historical OHLCV data for target ticker
etf_df = yf.download(
    "STW.AX, VAS.AX, SPY.AX", 
    period="max"
)

[*********************100%***********************]  3 of 3 completed


In [3]:
# Review data to see the date ranges
display(etf_df.head())
display(etf_df.tail())
display(etf_df.shape)
display(etf_df.index)


Adj Close                    Close                     High  \
              SPY.AX     STW.AX VAS.AX SPY.AX     STW.AX VAS.AX SPY.AX   
Date                                                                     
2008-01-02       NaN  61.169998    NaN    NaN  61.169998    NaN    NaN   
2008-01-03       NaN  60.590000    NaN    NaN  60.590000    NaN    NaN   
2008-01-04       NaN  60.740002    NaN    NaN  60.740002    NaN    NaN   
2008-01-07       NaN  59.349998    NaN    NaN  59.349998    NaN    NaN   
2008-01-08       NaN  59.020000    NaN    NaN  59.020000    NaN    NaN   

                                Low                     Open             \
               STW.AX VAS.AX SPY.AX     STW.AX VAS.AX SPY.AX     STW.AX   
Date                                                                      
2008-01-02  61.169998    NaN    NaN  61.169998    NaN    NaN  61.169998   
2008-01-03  60.590000    NaN    NaN  60.590000    NaN    NaN  60.590000   
2008-01-04  60.740002    NaN    NaN  60.740002    NaN    NaN  60.740002   
2008-01-07  59.349998    NaN    NaN  59.349998    NaN    NaN  59.349998   
2008-01-08  59.020000    NaN    NaN  59.020000    NaN    NaN  59.020000   

                  Volume                
           VAS.AX SPY.AX STW.AX VAS.AX  
Date                                    
2008-01-02    NaN    NaN      0    NaN  
2008-01-03    NaN    NaN      0    NaN  
2008-01-04    NaN    NaN      0    NaN  
2008-01-07    NaN    NaN      0    NaN  
2008-01-08    NaN    NaN      0    NaN

Adj Close                             Close             \
                SPY.AX     STW.AX     VAS.AX      SPY.AX     STW.AX   
Date                                                                  
2022-12-23  570.000000  64.610001  88.360001  570.000000  64.610001   
2022-12-28  566.070007  64.389999  88.169998  566.070007  64.389999   
2022-12-29  559.520020  63.320000  87.440002  559.520020  63.320000   
2022-12-30  565.000000  63.509998  87.699997  565.000000  63.509998   
2023-01-03  561.260010  62.689999  85.809998  561.260010  62.689999   

                             High                               Low  \
               VAS.AX      SPY.AX     STW.AX     VAS.AX      SPY.AX   
Date                                                                  
2022-12-23  88.360001  573.030029  64.629997  88.389999  569.900024   
2022-12-28  88.169998  575.000000  64.800003  88.610001  566.070007   
2022-12-29  87.440002  560.609985  64.029999  87.820000  558.599976   
2022-12-30  87.699997  566.179993  63.840000  88.129997  565.000000   
2023-01-03  85.809998  567.020020  63.849998  87.650002  561.260010   

                                        Open                       Volume  \
               STW.AX     VAS.AX      SPY.AX     STW.AX     VAS.AX SPY.AX   
Date                                                                        
2022-12-23  64.220001  87.820000  573.030029  64.610001  88.339996  103.0   
2022-12-28  64.279999  87.879997  570.000000  64.750000  88.500000  562.0   
2022-12-29  63.119999  87.089996  560.609985  64.029999  87.820000  566.0   
2022-12-30  63.509998  87.699997  565.500000  63.720001  87.930000  152.0   
2023-01-03  62.299999  85.239998  567.020020  63.830002  87.650002  162.0   

                              
            STW.AX    VAS.AX  
Date                          
2022-12-23  110893  224131.0  
2022-12-28   61408  759834.0  
2022-12-29  181484  280709.0  
2022-12-30   30139   96586.0  
2023-01-03  165572  295130.0

(3795, 18)

DatetimeIndex(['2008-01-02', '2008-01-03', '2008-01-04', '2008-01-07',
               '2008-01-08', '2008-01-09', '2008-01-10', '2008-01-11',
               '2008-01-14', '2008-01-15',
               ...
               '2022-12-16', '2022-12-19', '2022-12-20', '2022-12-21',
               '2022-12-22', '2022-12-23', '2022-12-28', '2022-12-29',
               '2022-12-30', '2023-01-03'],
              dtype='datetime64[ns]', name='Date', length=3795, freq=None)

In [4]:
# Examine the column type and check for nulls
etf_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3795 entries, 2008-01-02 to 2023-01-03
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   (Adj Close, SPY.AX)  2079 non-null   float64
 1   (Adj Close, STW.AX)  3795 non-null   float64
 2   (Adj Close, VAS.AX)  3458 non-null   float64
 3   (Close, SPY.AX)      2079 non-null   float64
 4   (Close, STW.AX)      3795 non-null   float64
 5   (Close, VAS.AX)      3458 non-null   float64
 6   (High, SPY.AX)       2079 non-null   float64
 7   (High, STW.AX)       3795 non-null   float64
 8   (High, VAS.AX)       3458 non-null   float64
 9   (Low, SPY.AX)        2079 non-null   float64
 10  (Low, STW.AX)        3795 non-null   float64
 11  (Low, VAS.AX)        3458 non-null   float64
 12  (Open, SPY.AX)       2079 non-null   float64
 13  (Open, STW.AX)       3795 non-null   float64
 14  (Open, VAS.AX)       3458 non-null   float64
 15  (Volume, SPY.AX)    

In [5]:
# Retain the key columns for each etf
etf_df = etf_df.drop(columns = ['Adj Close', 'High', 'Low', 'Open', 'Volume'])
etf_df.columns = etf_df.columns.droplevel()     # Changes the multilevel indexing on columns to single level


In [6]:
etf_df.head()

,SPY.AX,STW.AX,VAS.AX
Date,,,
2008-01-02,NaN,61.169998,NaN
2008-01-03,NaN,60.590000,NaN
2008-01-04,NaN,60.740002,NaN
2008-01-07,NaN,59.349998,NaN
2008-01-08,NaN,59.020000,NaN


In [7]:
#  Drop all nulls
etf_df = etf_df.dropna()
etf_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2078 entries, 2014-10-13 to 2023-01-03
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   SPY.AX  2078 non-null   float64
 1   STW.AX  2078 non-null   float64
 2   VAS.AX  2078 non-null   float64
dtypes: float64(3)
memory usage: 64.9 KB


In [8]:
etf_df.head()

,SPY.AX,STW.AX,VAS.AX
Date,,,
2014-10-13,216.729996,48.599998,65.360001
2014-10-14,213.949997,49.119999,66.059998
2014-10-15,217.089996,49.500000,66.480003
2014-10-16,212.434738,49.549999,66.599998
2014-10-17,213.059998,49.779999,66.839996


In [68]:
# Downloading news items related to a specific stock
spy_df = yf.Ticker('SPY.AX')


TypeError: 'dict' object is not callable

In [70]:
print(spy_df.info)

{'exchange': 'ASX', 'shortName': 'SPDR 500 CDI 1:1', 'longName': 'SPDR S&P 500 ETF Trust', 'exchangeTimezoneName': 'Australia/Sydney', 'exchangeTimezoneShortName': 'AEDT', 'isEsgPopulated': False, 'gmtOffSetMilliseconds': '39600000', 'quoteType': 'ETF', 'symbol': 'SPY.AX', 'messageBoardId': 'finmb_6160262', 'market': 'au_market', 'annualHoldingsTurnover': None, 'enterpriseToRevenue': None, 'beta3Year': 0.97, 'profitMargins': None, 'enterpriseToEbitda': None, '52WeekChange': None, 'morningStarRiskRating': None, 'forwardEps': None, 'revenueQuarterlyGrowth': None, 'fundInceptionDate': 727660800, 'annualReportExpenseRatio': None, 'totalAssets': 139454240, 'bookValue': None, 'fundFamily': 'State Street Global Advisors (Aus) Ltd', 'lastFiscalYearEnd': None, 'netIncomeToCommon': None, 'trailingEps': None, 'lastDividendValue': None, 'SandP52WeekChange': None, 'priceToBook': None, 'nextFiscalYearEnd': None, 'yield': 0.0114, 'mostRecentQuarter': None, 'enterpriseValue': None, 'priceHint': 2, 'th

In [21]:
type(spy_df.news)

list

In [11]:
len(spy_df.news)

8

In [12]:
spy_df.news[5]

{'uuid': '3b833990-c849-3271-bdd3-13d559454cb3',
 'title': 'Damar Hamlin of the Buffalo Bills Is in Critical Condition After Collapsing on the Field',
 'publisher': 'The Wall Street Journal',
 'link': 'https://finance.yahoo.com/m/3b833990-c849-3271-bdd3-13d559454cb3/damar-hamlin-of-the-buffalo.html',
 'providerPublishTime': 1672719780,
 'type': 'STORY',
 'thumbnail': {'resolutions': [{'url': 'https://s.yimg.com/uu/api/res/1.2/pYIMZ248kCgtS21UV3bsVQ--~B/aD02NDA7dz0xMjgwO2FwcGlkPXl0YWNoeW9u/https://media.zenfs.com/en/wsj.com/38c8389e5057f483e7b24337525fbda0',
    'width': 1280,
    'height': 640,
    'tag': 'original'},
   {'url': 'https://s.yimg.com/uu/api/res/1.2/NTOPm.6LK1WhpnFUOfSUmA--~B/Zmk9ZmlsbDtoPTE0MDtweW9mZj0wO3c9MTQwO2FwcGlkPXl0YWNoeW9u/https://media.zenfs.com/en/wsj.com/38c8389e5057f483e7b24337525fbda0',
    'width': 140,
    'height': 140,
    'tag': '140x140'}]}}

In [13]:
# Collect the top 10 stocks of the S&P 500 to use as proxy for the SPY.AX ETF
# Note - hardcoding the stocks as there does not appear to be an easier way to 
# programitcally retrieve the S&P500 constituent stocks
spy_top10 = [
    'AAPL', 'MSFT', 'AMZN', 'TSLA', 'GOOGL', 'GOOG', 'NVDA', 'BRK.B', 'META', 'UNH'
]

In [14]:
# Retrieve alpaca credentials
load_dotenv()
alpaca_api_key = os.getenv("APCA-API-KEY-ID")
alpaca_secret_key = os.getenv("APCA-API-SECRET-KEY")
if not(alpaca_api_key) or not(alpaca_secret_key):
    print("Failed to load API credentials")

In [15]:
# Prepare to retrieve news from alpaca
alpaca_endpoint = 'https://data.alpaca.markets/v1beta1/news'

alpaca_headers = {
    'Apca-Api-Key-Id': alpaca_api_key,
    'Apca-Api-Secret-Key': alpaca_secret_key
}


In [19]:
# Get the news for each of spy's top 10 stocks
parameters = {
    'symbols': spy_top10[0],
    'start': pd.to_datetime(etf_df.index[0]).strftime('%Y-%m-%d'),
    'end': pd.to_datetime(etf_df.index[-1]).strftime('%Y-%m-%d'),
    'limit': 50,
    'include_content': False,
}

response = requests.get(
    url = alpaca_endpoint,
    headers = alpaca_headers,
    params = parameters
)

response.raise_for_status()
display(response.json())

{'news': [{'author': 'Michael Cohen',
   'content': '',
   'created_at': '2022-12-31T19:03:10Z',
   'headline': 'Bulls And Bears Of The Week: Apple, AMC, Meta, And How Elon Musk Is Reacting To Tesla Stock Plunge',
   'id': 30242858,
   'images': [{'size': 'large',
     'url': 'https://cdn.benzinga.com/files/imagecache/2048x1536xUP/images/story/2022/12/30/bulls_bears5.png'},
    {'size': 'small',
     'url': 'https://cdn.benzinga.com/files/imagecache/1024x768xUP/images/story/2022/12/30/bulls_bears5.png'},
    {'size': 'thumb',
     'url': 'https://cdn.benzinga.com/files/imagecache/250x187xUP/images/story/2022/12/30/bulls_bears5.png'}],
   'source': 'benzinga',
   'summary': 'Benzinga\xa0examined the prospects for many investors&#39;\xa0favorite stocks over the last week — here&#39;s a look at some of our top stories.\xa0',
   'symbols': ['AAPL', 'AMC', 'BTCUSD', 'LUV', 'META', 'MSTR', 'TSLA'],
   'updated_at': '2023-01-01T02:10:43Z',
   'url': 'https://www.benzinga.com/news/large-cap/22

In [34]:
response.json()['news'][-1]
spy_news = pd.DataFrame(response.json()['news'])
spy_news.head()

,author,content,created_at,headline,id,images,source,summary,symbols,updated_at,url
0,Michael Cohen,,2022-12-31T19:03:10Z,"Bulls And Bears Of The Week: Apple, AMC, Meta,...",30242858,"[{'size': 'large', 'url': 'https://cdn.benzing...",benzinga,Benzinga examined the prospects for many inves...,"[AAPL, AMC, BTCUSD, LUV, META, MSTR, TSLA]",2023-01-01T02:10:43Z,https://www.benzinga.com/news/large-cap/22/12/...
1,Shanthi Rexaline,,2022-12-31T16:11:14Z,Bulls In A Bear Market: These 10 Stocks Clocke...,30212759,"[{'size': 'large', 'url': 'https://cdn.benzing...",benzinga,2022 would go down as one of the worst years f...,"[AAPL, ADMA, AMPY, ARDX, CTIC, HDSN, HP, INSW,...",2022-12-31T16:11:14Z,https://www.benzinga.com/analyst-ratings/analy...
2,Charles Gross,,2022-12-30T17:52:25Z,Top Wall Street Bets Mentions for Friday Decem...,30239495,[],benzinga,,"[AAPL, AMC, AMZN, ASTS, GME, JPM, NFLX, NIO, N...",2022-12-30T17:52:25Z,https://www.benzinga.com/news/22/12/30239495/t...
3,Benzinga Newsdesk,,2022-12-30T16:54:01Z,'Tim Cook relayed concern over app store curbs...,30238792,[],benzinga,,[AAPL],2022-12-30T16:54:02Z,https://www.benzinga.com/news/22/12/30238792/t...
4,Anusuya Lahiri,,2022-12-30T14:33:32Z,"Huawei's Business Comes Out Of The Woods, Chai...",30235878,"[{'size': 'large', 'url': 'https://cdn.benzing...",benzinga,\n,"[AAPL, SSNLF]",2022-12-30T14:34:33Z,https://www.benzinga.com/news/earnings/22/12/3...


In [65]:
# Loop through all the remaining news data
# while(response.json()['next_page_token'] != ''):
parameters = {
    'symbols': spy_top10[0],
    'start': pd.to_datetime(etf_df.index[0]).strftime('%Y-%m-%d'),
    'end': pd.to_datetime(etf_df.index[-1]).strftime('%Y-%m-%d'),
    'limit': 50,
    'include_content': False,
    'page_token': response.json()['next_page_token']
}

response = requests.get(
    url = alpaca_endpoint,
    headers = alpaca_headers,
    params = parameters
)

response.raise_for_status()

temp_df = pd.DataFrame(response.json()['news'])
temp_df = temp_df.loc[temp_df['headline'].str.contains('apple', na=False, case=False)]


In [66]:
# Split the index into date and time
temp_df.index = pd.to_datetime(temp_df['updated_at'])
temp_df.index = pd.MultiIndex.from_arrays([temp_df.index.date, temp_df.index.time], names=['Date','Time'])
temp_df.head()

author content            created_at  \
Date       Time                                                        
2021-10-05 18:54:22        Randy Elias          2021-10-05T18:35:42Z   
           17:08:39  Benzinga Newsdesk          2021-10-05T17:08:39Z   
           15:57:08  Benzinga Newsdesk          2021-10-05T15:57:07Z   
           14:31:33        Adam Eckert          2021-10-05T14:31:33Z   
           08:45:36        Rachit Vats          2021-10-05T08:45:36Z   

                                                              headline  \
Date       Time                                                          
2021-10-05 18:54:22                        Why Apple Shares Are Rising   
           17:08:39  'Apple Pay lands in Bahrain with three major b...   
           15:57:08  Visa To Change Way It Processes Certain Apple ...   
           14:31:33  Is Now The Time To Buy The Dip In Apple And Mi...   
           08:45:36  Apple Taken To Court By Ericsson Over Allegati...   

                           id  \
Date       Time                 
2021-10-05 18:54:22  23247353   
           17:08:39  23245806   
           15:57:08  23244310   
           14:31:33  23240861   
           08:45:36  23233901   

                                                                images source  \
Date       Time                                                                 
2021-10-05 18:54:22  [{'size': 'large', 'url': 'https://cdn.benzing...          
           17:08:39                                                 []          
           15:57:08                                                 []          
           14:31:33  [{'size': 'large', 'url': 'https://cdn.benzing...          
           08:45:36  [{'size': 'large', 'url': 'https://cdn.benzing...          

                                                               summary  \
Date       Time                                                          
2021-10-05 18:54:22  Shares of companies in the broader technology ...   
           17:08:39                                                      
           15:57:08                                                      
           14:31:33  With the Nasdaq down more than 7.5% from its h...   
           08:45:36  Ericsson AB (NASDAQ: ERIC) on Monday filed a l...   

                          symbols            updated_at  \
Date       Time                                           
2021-10-05 18:54:22        [AAPL]  2021-10-05T18:54:22Z   
           17:08:39        [AAPL]  2021-10-05T17:08:39Z   
           15:57:08     [AAPL, V]  2021-10-05T15:57:08Z   
           14:31:33  [AAPL, MSFT]  2021-10-05T14:31:33Z   
           08:45:36  [AAPL, ERIC]  2021-10-05T08:45:36Z   

                                                                   url  
Date       Time                                                         
2021-10-05 18:54:22  https://www.benzinga.com/news/21/10/23247353/w...  
           17:08:39  https://www.benzinga.com/news/21/10/23245806/a...  
           15:57:08  https://www.benzinga.com/news/21/10/23244310/v...  
           14:31:33  https://www.benzinga.com/analyst-ratings/analy...  
           08:45:36  https://www.benzinga.com/news/21/10/23233901/a...

In [71]:
from transformers import pipeline


In [74]:
etf_pipeline = pipeline('sentiment-analysis')


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [91]:
# Transform a headline into a sentiment
def get_sentiment(headline):
    return etf_pipeline(headline)[0]

Why Apple Shares Are Rising
{'label': 'POSITIVE', 'score': 0.9917313456535339}


In [79]:
print(temp_df.iloc[4]['headline'])
etf_pipeline(temp_df.iloc[4]['headline'])

Apple Taken To Court By Ericsson Over Allegations Of Engaging In Unfair Practices In 5G Patent Royalty Talks


[{'label': 'NEGATIVE', 'score': 0.9977689981460571}]

In [81]:
print(temp_df.iloc[3]['headline'])
etf_pipeline(temp_df.iloc[3]['headline'])

Is Now The Time To Buy The Dip In Apple And Microsoft?


[{'label': 'NEGATIVE', 'score': 0.9965846538543701}]

In [82]:
print(temp_df.iloc[1]['headline'])
etf_pipeline(temp_df.iloc[1]['headline'])

'Apple Pay lands in Bahrain with three major banks supported at launch' -Apple Insider Report


[{'label': 'NEGATIVE', 'score': 0.8031097054481506}]

In [84]:
etf_pipeline('Apple Pay lands in Bahrain with three major banks supported at launch')

[{'label': 'POSITIVE', 'score': 0.9262937903404236}]

In [83]:
print(temp_df.iloc[2]['headline'])
etf_pipeline(temp_df.iloc[2]['headline'])

Visa To Change Way It Processes Certain Apple Pay Transactions, Change Will Trim Fees Banks Pay To Apple On Recurring Transactions


[{'label': 'NEGATIVE', 'score': 0.9922193288803101}]

In [86]:
print(temp_df.iloc[10]['headline'])
etf_pipeline(temp_df.iloc[10]['headline'])

Benzinga's Bulls And Bears Of The Week: Apple, Boeing, Moderna, Robinhood And More


[{'label': 'POSITIVE', 'score': 0.9982277750968933}]

In [17]:
stw_df = yf.Ticker('STW.AX')
stw_news = stw_df.news
len(stw_news)

8

In [20]:
stw_news[:8]

[{'uuid': '83d81499-1f54-322b-83f5-40cd7f4d0214',
  'title': 'Investors in Cliq Digital (ETR:CLIQ) have made a incredible return of 861% over the past three years',
  'publisher': 'Simply Wall St.',
  'link': 'https://finance.yahoo.com/news/investors-cliq-digital-etr-cliq-041349727.html',
  'providerPublishTime': 1672287229,
  'type': 'STORY',
  'thumbnail': {'resolutions': [{'url': 'https://s.yimg.com/uu/api/res/1.2/gnGTQ_q9KCdl7Xvx_ppB0g--~B/aD00MzI7dz0xMTk0O2FwcGlkPXl0YWNoeW9u/https://media.zenfs.com/en/simply_wall_st__316/0fe4d5d105492213ce106a92b49639d4',
     'width': 1194,
     'height': 432,
     'tag': 'original'},
    {'url': 'https://s.yimg.com/uu/api/res/1.2/ooX5AtoQCMx8g9kEgh.tsQ--~B/Zmk9ZmlsbDtoPTE0MDtweW9mZj0wO3c9MTQwO2FwcGlkPXl0YWNoeW9u/https://media.zenfs.com/en/simply_wall_st__316/0fe4d5d105492213ce106a92b49639d4',
     'width': 140,
     'height': 140,
     'tag': '140x140'}]}},
 {'uuid': 'd4095648-af2c-3f4c-8a38-3bc9a3830bfd',
  'title': 'NORMA Group SE (ETR:NOEJ) i

In [21]:
vas_df = yf.Ticker('VAS.AX')
vas_news = vas_df.news
vas_news

[{'uuid': '83d81499-1f54-322b-83f5-40cd7f4d0214',
  'title': 'Investors in Cliq Digital (ETR:CLIQ) have made a incredible return of 861% over the past three years',
  'publisher': 'Simply Wall St.',
  'link': 'https://finance.yahoo.com/news/investors-cliq-digital-etr-cliq-041349727.html',
  'providerPublishTime': 1672287229,
  'type': 'STORY',
  'thumbnail': {'resolutions': [{'url': 'https://s.yimg.com/uu/api/res/1.2/gnGTQ_q9KCdl7Xvx_ppB0g--~B/aD00MzI7dz0xMTk0O2FwcGlkPXl0YWNoeW9u/https://media.zenfs.com/en/simply_wall_st__316/0fe4d5d105492213ce106a92b49639d4',
     'width': 1194,
     'height': 432,
     'tag': 'original'},
    {'url': 'https://s.yimg.com/uu/api/res/1.2/ooX5AtoQCMx8g9kEgh.tsQ--~B/Zmk9ZmlsbDtoPTE0MDtweW9mZj0wO3c9MTQwO2FwcGlkPXl0YWNoeW9u/https://media.zenfs.com/en/simply_wall_st__316/0fe4d5d105492213ce106a92b49639d4',
     'width': 140,
     'height': 140,
     'tag': '140x140'}]}},
 {'uuid': 'd4095648-af2c-3f4c-8a38-3bc9a3830bfd',
  'title': 'NORMA Group SE (ETR:NOEJ) i

In [22]:
vas_news[-1]

{'uuid': '1e4bb96f-3899-3e11-a1bd-141924a4b16b',
 'title': 'Chinese Gaming Bonanza Ignites Tencent, Asian Gaming Rally',
 'publisher': 'Bloomberg',
 'link': 'https://finance.yahoo.com/news/chinese-gaming-bonanza-ignites-tencent-035410272.html',
 'providerPublishTime': 1672286050,
 'type': 'STORY',
 'thumbnail': {'resolutions': [{'url': 'https://s.yimg.com/uu/api/res/1.2/FC4IjAMF3HDWsWZj_78d8A--~B/aD0xMzM0O3c9MjAwMDthcHBpZD15dGFjaHlvbg--/https://media.zenfs.com/en/bloomberg_technology_68/7862c3e6e7a393efc2d3f0aae966b154',
    'width': 2000,
    'height': 1334,
    'tag': 'original'},
   {'url': 'https://s.yimg.com/uu/api/res/1.2/ehfL7nJI1CqbinDYBYTjqA--~B/Zmk9ZmlsbDtoPTE0MDtweW9mZj0wO3c9MTQwO2FwcGlkPXl0YWNoeW9u/https://media.zenfs.com/en/bloomberg_technology_68/7862c3e6e7a393efc2d3f0aae966b154',
    'width': 140,
    'height': 140,
    'tag': '140x140'}]},
 'relatedTickers': ['TCEHY']}

In [23]:
tsla_df = yf.Ticker('TSLA')
tsla_news = tsla_df.news
tsla_news

[{'uuid': '7718706f-6809-36f4-b662-6a8d8e4c3205',
  'title': 'Dow Jones, Nasdaq Break Key Levels As Apple Skids, Tesla Ends Dive; What To Do Now',
  'publisher': "Investor's Business Daily",
  'link': 'https://finance.yahoo.com/m/7718706f-6809-36f4-b662-6a8d8e4c3205/dow-jones%2C-nasdaq-break-key.html',
  'providerPublishTime': 1672288907,
  'type': 'STORY',
  'thumbnail': {'resolutions': [{'url': 'https://s.yimg.com/uu/api/res/1.2/mxVgBGHlXoOaKc5bmhSFUA--~B/aD01NjU7dz0xMDAwO2FwcGlkPXl0YWNoeW9u/https://media.zenfs.com/en/ibd.com/c858484a148a1a8104b2fdb9e68c2f4b',
     'width': 1000,
     'height': 565,
     'tag': 'original'},
    {'url': 'https://s.yimg.com/uu/api/res/1.2/yxTeuVj9n3HQjIEebHy46w--~B/Zmk9ZmlsbDtoPTE0MDtweW9mZj0wO3c9MTQwO2FwcGlkPXl0YWNoeW9u/https://media.zenfs.com/en/ibd.com/c858484a148a1a8104b2fdb9e68c2f4b',
     'width': 140,
     'height': 140,
     'tag': '140x140'}]},
  'relatedTickers': ['^DJI', 'COMP', '^GSPC', 'TSLA', 'AAPL', 'FSLR']},
 {'uuid': '85400be1-c6c8-324